In [1]:
import os
#imported os to minimize the size of text of the song name at statusbar...

from tkinter import * 
'''tkinter is for GUI, we have to import 'tkinter' to work on graphics... 
it's already installed we have to just call it and use it...'''

import tkinter.messagebox  # this is use to show any kind of music

from tkinter import filedialog #this is to browse and choose any file which we want to play...

from mutagen.mp3 import MP3 # This is for to control the flow of mp3 file...

import threading # This is to let the program look all over or don't get hang... 

import time # this is to use time_length and current Time label...

from tkinter import ttk

from ttkthemes import themed_tk as tk

from pygame import mixer 

'''pygame is for Music control, We have to use pygame it give to facility to control the music and all stufs... 
To use this we have to first pip install pygame or conda...'''

root=tk.ThemedTk() # This is for Frame or structure where we all stuffs put and work... 
root.get_themes()
root.set_theme("radiance")


menubar=Menu(root) # This is menubar 
root.config(menu=menubar) # This is to configure menu ,it will tell menu where to go or stand...

subMenu=Menu(menubar,tearoff=0) # This is submenu it's a part of menu where we will find all things like 'File'  

playlist=[]

def browse_file(): # This is to browse the particular file into sangeetholic...
    global filename_path # This is global variable to use anywhere in the program...
    filename_path=filedialog.askopenfilename() # this is to choose any file from the system
    add_to_playlist(filename_path)
    
def add_to_playlist(filename):
    filename=os.path.basename(filename)
    index=0
    playlistbox.insert(index,filename)
    playlist.insert(index,filename_path)
    playlistbox.pack()
    index+=1

menubar.add_cascade(label='file',underline=0,menu=subMenu) # This is for drop down menu
subMenu.add_command(label='Open',command=browse_file)
subMenu.add_separator()
subMenu.add_command(label='Exit',command=root.destroy,accelerator="Ctrl+x") # this is exit the window...
subMenu.add_separator()

# This a part of the cascade or drop down menu

def about_us(): # this is to show the information...in About us menu...
    tkinter.messagebox.showinfo('About Sangeetholic', 'This is a small project which is created by Ankit Pratap Singh')

subMenu=Menu(menubar,tearoff=0) # This is submenu it's a part of menu where we will find all things like 'File'  
menubar.add_cascade(label='Help',underline=0,menu=subMenu) # This is for drop down menu
subMenu.add_command(label='About us',command=about_us) 
subMenu.add_separator()


mixer.init() # To use pygame mixer class we have to first initialize the mixer to use it...

root.title('Sangeetaholic') # it's for title

root.iconbitmap(r'images/music.ico') # it's for icon which we found on the top left cornor it take .ioc extension photo...

#root.geometry('400x300') # this is for the size of the frame

# This is statusbar use to show the status in the sangeet ...
statusbar=ttk.Label(root, text='Welcome to Sangeetholic',relief=SUNKEN,anchor=W,font='ariel,10,bold')
statusbar.pack(side=BOTTOM,fill=X)

leftframe=Frame(root)
leftframe.pack(side=LEFT,padx=30)

playlistbox=Listbox(leftframe)
playlistbox.pack()

addbtn=ttk.Button(leftframe,text='+ Add',command=browse_file)
addbtn.pack(side=LEFT)

def delete_song():
    selected_song=playlistbox.curselection()
    selected_song=int(selected_song[0])
    playlistbox.delete(selected_song)
    playlist.pop(selected_song)
    
delbtn=ttk.Button(leftframe,text='- Delete',command=delete_song)
delbtn.pack(side=LEFT)

rightframe=Frame(root)
rightframe.pack()

topframe=Frame(rightframe)
topframe.pack()

lengthlabel=ttk.Label(topframe,text='Total Length - --:--',font='ariel,8,bold') # it's a label to get the length of the file...
lengthlabel.pack(pady=10)

currenttimelabel=ttk.Label(topframe,text='Time Left - --:--',relief=GROOVE,font='ariel,8,bold') # it's a label for current time 
currenttimelabel.pack()

#labelphoto=Label(root,image=photo)
#labelphoto.pack()

def show_details(play_song): # this is to control the time duration and get it and use it into progrm...
    
    file_data=os.path.splitext(play_song)# this will help us to distinguish between .wav and .mp3...
    
    if file_data[1]=='.mp3': # if file is mp3 them this portion will work...
        audio=MP3(play_song)
        total_length=audio.info.length
        
    else:
        a=mixer.sound(play_song) # if file is .wav then this will work...
        total_length=a.get_length()
        
    minu,sec = divmod(total_length,60)#minu and sec are variable which help divide all the timelength into minutes and second...
    minu=round(minu)# this will round the minutes
    sec=round(sec)#this will round second
    timeformat= '{:02d}:{:02d}'.format(minu,sec) # this is timeformat
    lengthlabel['text']='Total Length'+'---'+ timeformat
    
    t1=threading.Thread(target=start_count,args=(total_length,)) # here is the use of threading
    t1.start()
    

def start_count(t): # this is use to get how much time have left to end the music
    global paused
    while t and mixer.music.get_busy(): # this is to stop the timer
        if paused:
            continue # if music is not paused then passed to next iteration...
        else:
            minu,sec = divmod(t,60)
            minu=round(minu)
            sec=round(sec)
            timeformat= '{:02d}:{:02d}'.format(minu,sec)
            currenttimelabel['text']='Time Left'+'---'+ timeformat
            time.sleep(1)
            t-=1
    

def play_btn(): # this is for playButton to start the music
    global paused
    if paused:
        mixer.music.unpause()
        statusbar['text']='Music Resumed'
        paused=False
    else:
        try:
            stop_btn()
            time.sleep(1)
            selected_song=playlistbox.curselection()
            selected_song=int(selected_song[0])
            playit=playlist[selected_song]
            mixer.music.load(playit)# to play the music we first have to load it...
            mixer.music.play() # then play it
            statusbar['text']='Play Music'+'---'+os.path.basename(playit)
            show_details(playit)
            # statusbar use to short the name of music or remove pathname from status bar...
            
        except:
            tkinter.messagebox.showerror('File not found', 'Sangeetholic can not find the file, Please check it again')


paused = False

def pause_btn(): # This is for pause music...
    global paused  # declare pause a global variable...
    paused=True
    mixer.music.pause()
    statusbar['text']='Music Paused'
    

def rewind_btn():
    play_btn()
    statusbar['text']='Music Rewinded'

def stop_btn(): # This is for stop the music
    mixer.music.stop()
    statusbar['text']='Music Stopped'

def set_vol(vol): # this is for set the volume...
    volume=float(vol)/100 # set_volume of the mixer take values from 0 to 1...
    mixer.music.set_volume(volume)# this will take value from volume variable to set the music volume...

muted=False

def mute_btn(): # this is for mute the music... and change the image from mute to play vice-versa...
    global muted
    if muted:
        mixer.music.set_volume(0.3)
        volumebtn.configure(image=volumephoto)
        scale.set(30)
        muted=False
    else:
        mixer.music.set_volume(0)
        volumebtn.configure(image=mutephoto)
        scale.set(0)
        muted=True
    

middleframe=Frame(rightframe)
middleframe.pack(pady=10)
    
    
playphoto=PhotoImage(file='images/play.png') # This is for playmusic photo button
playbtn=ttk.Button(middleframe,image=playphoto,command=play_btn)
'''This is the button where we have describe where the button go(root), 
image which image we want to set(playphoto), command what to do(play_btn)'''
playbtn.grid(row=0,column=0,padx=10) # this is to pack the play button

    
stopphoto=PhotoImage(file='images/stop.png') # This is for stop button image...
stopbtn=ttk.Button(middleframe,image=stopphoto,command=stop_btn) # this is same as play button
stopbtn.grid(row=0,column=1,padx=10)

pausephoto=PhotoImage(file='images/pause.png') # This is for pause button image...
pausebtn=ttk.Button(middleframe,image=pausephoto,command=pause_btn) # this is same as pause button
pausebtn.grid(row=0,column=2,padx=10)


bottomframe=Frame(rightframe)
bottomframe.pack(pady=30,padx=30)

rewindphoto=PhotoImage(file='images/rewind.png') # This is for pause button image...
rewindbtn=ttk.Button(bottomframe,image=rewindphoto,command=rewind_btn) # this is same as pause button
rewindbtn.grid(row=0,column=0)

mutephoto=PhotoImage(file='images/mute.png')
volumephoto=PhotoImage(file='images/volume.png')# This is for pause button image...
volumebtn=ttk.Button(bottomframe,image=volumephoto,command=mute_btn) # this is same as pause button
volumebtn.grid(row=0,column=1)

''' This(sacle) is feature of mixer class which let us to set the parameter for volume, 
here we root which tell the scale where to go, from _ tells from where to start to tells where to stop, 
orient tell how to look into the frame vertical or horizontal (should be Capltal letters) and command tells what to do'''

scale=ttk.Scale(bottomframe, from_=0, to=100, orient=HORIZONTAL, command=set_vol) 
scale.set(50) 
mixer.music.set_volume(0.5) # This is for set the volume at a fix position always
scale.grid(row=0,column=2,pady=10,padx=30) # it's to pack the scale...



def on_closing():
    stop_btn()
    tkinter.messagebox.showinfo('Sangeetaholic','Do you want to close it ?')
    root.destroy()
root.protocol("WM_DELETE_WINDOW",on_closing)


root.mainloop() # this is let the frame to be in infinte loop if we don't do this then we will not able to see the frame...

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
